<a href="https://colab.research.google.com/github/tushant-akar/CS367-Artifical-Intelligence-Lab/blob/main/K_Sat_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import random
import time
from typing import List, Tuple, Callable

In [13]:
def generate_ksat_problem(k, m, n):
    formula = []

    for _ in range(m):
        # Generate a new clause
        clause = []
        available_vars = list(range(1, n + 1))

        for _ in range(k):
            if not available_vars:
                break

            # Choose a random variable from the available ones
            var = random.choice(available_vars)
            available_vars.remove(var)

            # Randomly negate the variable or not
            if random.random() < 0.5:
                var = -var

            clause.append(var)

        formula.append(clause)

    return formula

In [14]:
def print_formula(formula):
    print("Formula:")
    for i, clause in enumerate(formula, 1):
        clause_str = " ∨ ".join(map(str, clause))
        print(f"  ({clause_str})")
    print()

In [15]:
def heuristic1(assignment: List[bool], formula: List[List[int]]) -> int:
    return sum(any(literal > 0 and assignment[abs(literal) - 1] or
                   literal < 0 and not assignment[abs(literal) - 1]
                   for literal in clause)
               for clause in formula)

def heuristic2(assignment: List[bool], formula: List[List[int]]) -> int:
    return sum(2 ** sum(literal > 0 and assignment[abs(literal) - 1] or
                        literal < 0 and not assignment[abs(literal) - 1]
                        for literal in clause)
               for clause in formula)

In [16]:
def hill_climbing(formula: List[List[int]], n: int, heuristic: Callable, max_iterations: int = 1000) -> Tuple[List[bool], int]:
    assignment = [random.choice([True, False]) for _ in range(n)]
    best_score = heuristic(assignment, formula)

    for _ in range(max_iterations):
        improved = False
        for i in range(n):
            new_assignment = assignment.copy()
            new_assignment[i] = not new_assignment[i]
            new_score = heuristic(new_assignment, formula)
            if new_score > best_score:
                assignment = new_assignment
                best_score = new_score
                improved = True
                break
        if not improved:
            break

    return assignment, best_score

In [17]:
def beam_search(formula: List[List[int]], n: int, heuristic: Callable, beam_width: int, max_iterations: int = 1000) -> Tuple[List[bool], int]:
    beam = [[random.choice([True, False]) for _ in range(n)] for _ in range(beam_width)]
    best_assignment = None
    best_score = -1

    for _ in range(max_iterations):
        candidates = []
        for assignment in beam:
            for i in range(n):
                new_assignment = assignment.copy()
                new_assignment[i] = not new_assignment[i]
                score = heuristic(new_assignment, formula)
                candidates.append((new_assignment, score))

        candidates.sort(key=lambda x: x[1], reverse=True)
        beam = [candidate[0] for candidate in candidates[:beam_width]]

        if candidates[0][1] > best_score:
            best_assignment, best_score = candidates[0]

    return best_assignment, best_score

In [18]:
def vnd(formula: List[List[int]], n: int, heuristic: Callable, neighborhood_functions: List[Callable], max_iterations: int = 1000) -> Tuple[List[bool], int]:
    assignment = [random.choice([True, False]) for _ in range(n)]
    best_score = heuristic(assignment, formula)

    for _ in range(max_iterations):
        improved = False
        for nf in neighborhood_functions:
            new_assignment = nf(assignment)
            new_score = heuristic(new_assignment, formula)
            if new_score > best_score:
                assignment = new_assignment
                best_score = new_score
                improved = True
                break
        if not improved:
            break

    return assignment, best_score

# Neighborhood functions for VND
def flip_one(assignment: List[bool]) -> List[bool]:
    new_assignment = assignment.copy()
    i = random.randint(0, len(assignment) - 1)
    new_assignment[i] = not new_assignment[i]
    return new_assignment

def flip_two(assignment: List[bool]) -> List[bool]:
    new_assignment = assignment.copy()
    i, j = random.sample(range(len(assignment)), 2)
    new_assignment[i] = not new_assignment[i]
    new_assignment[j] = not new_assignment[j]
    return new_assignment

def swap_two(assignment: List[bool]) -> List[bool]:
    new_assignment = assignment.copy()
    i, j = random.sample(range(len(assignment)), 2)
    new_assignment[i], new_assignment[j] = new_assignment[j], new_assignment[i]
    return new_assignment

In [19]:
def calculate_penetrance(scores: List[int], max_score: int) -> float:
    return sum(score == max_score for score in scores) / len(scores)

In [20]:
def compare_algorithms(m: int, n: int, num_problems: int = 10, max_iterations: int = 500):
    algorithms = [
        ("Hill Climbing", lambda f, n, h: hill_climbing(f, n, h, max_iterations)),
        ("Beam Search (w=3)", lambda f, n, h: beam_search(f, n, h, 3, max_iterations)),
        ("Beam Search (w=4)", lambda f, n, h: beam_search(f, n, h, 4, max_iterations)),
        ("VND", lambda f, n, h: vnd(f, n, h, [flip_one, flip_two, swap_two], max_iterations))
    ]

    heuristics = [
        ("Heuristic 1", heuristic1),
        ("Heuristic 2", heuristic2)
    ]

    results = {(alg_name, heur_name): [] for alg_name, _ in algorithms for heur_name, _ in heuristics}

    for _ in range(num_problems):
        formula = generate_ksat_problem(3, m, n)
        max_score = m  # Maximum possible score is the number of clauses

        for (alg_name, algorithm), (heur_name, heuristic) in [(a, h) for a in algorithms for h in heuristics]:
            start_time = time.time()
            _, score = algorithm(formula, n, heuristic)
            elapsed_time = time.time() - start_time
            results[(alg_name, heur_name)].append((score, elapsed_time))

    print(f"Results for m={m}, n={n}:")
    for (alg_name, heur_name), scores_times in results.items():
        scores, times = zip(*scores_times)
        avg_score = sum(scores) / len(scores)
        avg_time = sum(times) / len(times)
        penetrance = calculate_penetrance(scores, max_score)
        print(f"  {alg_name} with {heur_name}:")
        print(f"    Average Score: {avg_score:.2f}")
        print(f"    Average Time: {avg_time:.4f} seconds")
        print(f"    Penetrance: {penetrance:.2%}")
        print()

In [21]:
if __name__ == "__main__":
    m_values = [25, 75, 150]
    n_values = [10, 43, 78]

    for m in m_values:
        for n in n_values:
            compare_algorithms(m, n)
            print("=" * 50)

Results for m=25, n=10:
  Hill Climbing with Heuristic 1:
    Average Score: 24.80
    Average Time: 0.0004 seconds
    Penetrance: 80.00%

  Hill Climbing with Heuristic 2:
    Average Score: 112.60
    Average Time: 0.0012 seconds
    Penetrance: 0.00%

  Beam Search (w=3) with Heuristic 1:
    Average Score: 24.90
    Average Time: 0.3137 seconds
    Penetrance: 90.00%

  Beam Search (w=3) with Heuristic 2:
    Average Score: 113.20
    Average Time: 0.4612 seconds
    Penetrance: 0.00%

  Beam Search (w=4) with Heuristic 1:
    Average Score: 25.00
    Average Time: 0.4236 seconds
    Penetrance: 100.00%

  Beam Search (w=4) with Heuristic 2:
    Average Score: 113.20
    Average Time: 0.6042 seconds
    Penetrance: 0.00%

  VND with Heuristic 1:
    Average Score: 23.60
    Average Time: 0.0002 seconds
    Penetrance: 10.00%

  VND with Heuristic 2:
    Average Score: 95.90
    Average Time: 0.0002 seconds
    Penetrance: 0.00%

Results for m=25, n=43:
  Hill Climbing with Heurist